In [1]:
!pip install requests
!pip install beautifulsoup4

In [2]:
import pandas as pd
import numpy as np
import requests
import urllib
import re
from bs4 import BeautifulSoup
import bs4

In [3]:
df_news = pd.read_csv('data/Macquarie_19900101-20220420.csv')

In [4]:
df_news

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,2.100601e+06,20220420,한국경제,김익환,대박 노리다 '700억' 물리더니 투자 스타일 확 바뀐 회사 [김익환의 컴퍼니워치],경제>금융_재테크,경제>증권_증시,경제>무역,NaN,NaN,NaN,"양홍석,양,이어룡,포메이션8,구본웅","미국,대주주,싱가포르,서울시,경기도","마음그룹,SC,LS그룹,예스코홀딩스,국민연,맥쿼리인프라,예스코,LS,대신증권,투자은행","700억,투자,스타일,회사,김익환,컴퍼니워치,LS,그룹,계열,도시가스,업체,예스코홀...","대신증권,예스코홀딩스,고배당주,맥쿼리인프라,도시가스,스타트업,부회장,700억,투자금...",LS그룹 계열의 도시가스업체 예스코홀딩스가 대신증권과 맥쿼리인프라를 비롯한 고배당주...,https://www.hankyung.com/economy/article/20220...,NaN
1,2.100101e+06,20220415,매일경제,신화,"미래에셋 리츠 ETF, 순자산 2000억 돌파",경제>증권_증시,경제>금융_재테크,경제>유통,NaN,NaN,NaN,NaN,NaN,"SK리츠,맥쿼리인프라,미래에셋,미래에셋자산운용,제이알글로벌리츠,ESR켄달스퀘어리츠,...","ETF,미래에셋,리츠,돌파,자산,미래에셋자산운용,TIGER,리츠부동산인프라,상장지수...","리츠부동산인프라,etf,2000억,원어치,미래에셋,인프라스트럭처,켄달스퀘어리츠,인컴...",미래에셋자산운용이 'TIGER 리츠부동산인프라' 상장지수펀드(ETF)의 순자산이 2...,http://news.mk.co.kr/newsRead.php?no=339384&ye...,NaN
2,1.100101e+06,20220414,경향신문,강연주 기자 play@kyunghyang.com,"김대기, 사외 감독이사 맡아 최소 6억 수령 ‘관피아’ 논란",정치>청와대,NaN,NaN,NaN,NaN,NaN,"김,이명박,김대기,권오인,한덕수,이창양",NaN,"경향신문,맥쿼리인프라,김앤장,두산중공업,SK하이닉스,현대두산인프라코어,현대두산인프라...","김대기,사외,감독이사,수령,최소,관피아,논란,2015년,SK이노베이션,재직,한화생명...","내정자,사외이사,두산중공업,한화생명,맥쿼리인프라,두산인프라코어,sk이노베이션,공무원...",2015년부터 SK이노베이션 현대두산인프라코어 등서 재직\n한화생명에선 ‘고문’으로...,https://www.khan.co.kr/national/national-gener...,NaN
3,1.100101e+06,20220414,경향신문,강연주 기자 play@kyunghyang.com,"[단독]김대기, 민간 사외이사 지내며 ‘최소 6억4100만원’ 수령 ‘관피아’ 논란",정치>청와대,NaN,NaN,NaN,NaN,NaN,"김,이명박,김대기,권오인,한덕수,이창양",NaN,"경향신문,맥쿼리인프라,김앤장,두산중공업,경제정의실천연합,현대두산인프라코어,맥쿼리한국...","김대기,민간,사외,이사,최소,수령,관피아,논란,김대기,대통령,비서실장,내정자,201...","내정자,사외이사,두산중공업,한화생명,sk이노베이션,이사회,두산인프라코어,맥쿼리인프라...",김대기 대통령 비서실장 내정자가 2015년부터 현재까지 민간 기업 사외 감독이사 활...,https://www.khan.co.kr/national/national-gener...,NaN
4,1.101001e+06,20220414,한겨레,박종오 기자,"김대기 비서실장, 바로 그 ‘맥쿼리’ 이사였다",정치>청와대,NaN,NaN,NaN,NaN,NaN,"김,노무현,이명박,김대기,윤증현,한승수,7천만원","인천김포고속도로,종로구,광주시,서울,9호선,통의동,서울지하철,서울시,부산신항,광주","에스케이이노베이션,인수위,맥쿼리인프라,대한상사중재원,서라벌도시가스,맥쿼리,두산중공업...","김대기,비서실장,맥쿼리,이사,김대기,대통령,비서실장,내정자,13일,민자,사업,투자,...","내정자,맥쿼리,이명박,맥쿼리인프라,민자사업,사외이사,한화생명,민자,윤증현,에스케이이...",김대기 대통령 비서실장 내정자가 13일 현재 민자사업 투자를 주력으로 하는 맥쿼리한...,http://www.hani.co.kr/arti/politics/politics_g...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022,1.100901e+06,20060221,중앙일보,NaN,[브리핑] 인프라 펀드 상장 규모 11조원 넘을 듯,경제>증권_증시,국제>유럽_EU,경제>금융_재테크,NaN,NaN,NaN,NaN,"미국,환산,한국,호주,런던","신한금융그룹,매쿼리은행,메릴린치,투자은행,MKIF","11조,인프라,펀드,상장,규모,투자은행,호주,최대,투자,은행,매쿼리은행,신한금융그룹...","1조,한국,다음달,런던,메릴린치,mkif,매쿼리,호주,11조,11억,gdr,신한금융...",호주 최대 투자은행인 매쿼리은행과 신한금융그룹이 공동 설립한 인프라 펀드의 한국.런...,https://news.joins.com/article/2210809,예외
1023,2.100601e+06,20060209,한국경제,NaN,"CLSA,""인프라펀드 상장 관련 건설주 주목""",경제>금융_재테크,경제>증권_증시,경제>산업_기업,NaN,NaN,NaN,NaN,NaN,"CL,정부,크레디리요네(CLSA)증권,한경닷컴,PF","CLSA,인프라펀드,상장,상장,건설,주목,크레디리요네,CLSA,증권,맥쿼리인프라펀드...","건설주,크레디리요네,대림산업,현대산업개발,기투자,맥쿼리인프라펀드,기록중,인프라펀드,...",9일 크레디리요네(CLSA)증권은 맥쿼리인프라펀드(MKIF) 상장 추진으로 사모펀드...,NaN,NaN
1024,2.100601e+06,20060207,한국경제,NaN,"맥쿼리 인프라펀드, 韓ㆍ英 동시상장 추진",경제>증권_증시,경제>산업_기업,경제>국제경제,NaN,NaN,NaN,NaN,"한국,영국,항만,증시,런던","금융감독원,맥쿼리한국인프,맥쿼리,라투융자회사,MKIF","맥쿼리,인프라,펀드,추진,韓ㆍ英,동시,상장,도로,항만,사회간접자본,SOC,시설,투자...","맥쿼리,mkif,영국,런던,인프라펀드,사회간접자본,라투,감독원,라투융자회사,한국인프...",도로 항만 등 사회간접자본(SOC)시설에 투자하는 인프라펀드인 맥쿼리한국인프\n라투...,NaN,NaN
1025,1.100901e+06,20060207,중앙일보,NaN,[브리핑] 신한-맥쿼리 은행 펀드 한 영서 상장,경제>금융_재테크,경제>증권_증시,경제>국제경제,NaN,NaN,NaN,NaN,"서울,천안,호주,런던,인천공항","신한금융그룹,금융감독원,맥쿼리 은행,맥쿼리,맥쿼리코리아,MKIF","신한,맥쿼리,은행,펀드,영서,상장,금융감독원,신한금융그룹,호주,최대,투자,은행,맥쿼...","고속도로,신한금융그룹,호주,맥쿼리코리아,mkif,인프라스트럭처,천안,논산,감독원,맥...",금융감독원은 7일 신한금융그룹과 호주 최대 투자 은행인 맥쿼리 은행이 공동 설립한 ...,https://news.joins.com/article/2137057,예외


In [5]:
#Uncomment to export the original dataframe into the excel format
#df_news.to_excel("output.xlsx")

In [6]:
#Set as :20 to make a small sample
df_news = df_news.iloc[:]

In [7]:
pd.set_option('display.max_colwidth', None)
df_news['URL'].astype(str)
df_news['URL'].fillna('nan')

0                         https://www.hankyung.com/economy/article/202204208696i
1                          http://news.mk.co.kr/newsRead.php?no=339384&year=2022
2       https://www.khan.co.kr/national/national-general/article/202204142115005
3       https://www.khan.co.kr/national/national-general/article/202204141105001
4              http://www.hani.co.kr/arti/politics/politics_general/1038853.html
                                          ...                                   
1022                                      https://news.joins.com/article/2210809
1023                                                                         nan
1024                                                                         nan
1025                                      https://news.joins.com/article/2137057
1026                                                                         nan
Name: URL, Length: 1027, dtype: object

In [8]:
# Loop through URLs, scrap information, identify the main body
# URL이 주어졌을 경우 해당 매체 웹사이트로부터 뉴스기사전문 크롤링

#최종결과물에 대한 텍스트 전처리-불용어 제거
#반복적인 불용어에 대한 메뉴얼 수정 내용 추가 - 미디어 매채별 예외처리 등록
def clean_html(raw_html):
    cleanr = re.compile('<.*?>|\s{2}|\n')
#    cleanr = re.compile('[^-가-힣a-zA-Z0-9/ ]')
    clean_text = re.sub(cleanr, '', raw_html)
    clean_text = clean_text.replace('window.jQuery || document.write("")', '')
    clean_text = clean_text.replace('Copyright ⓒ 동아일보 &amp; donga.com', '') #동아일보
    clean_text = clean_text.replace('좋아요 이미지좋아요슬퍼요 이미지슬퍼요화나요 이미지화나요후속기사 원해요 이미지후속기사 원해요기사추천 이미지기사추천공유 이미지공유공유하기닫기', '')
    clean_text = clean_text.replace('ⓒ 매일경제 &amp; mk.co.kr', '').replace('googletag.display("google_dfp_MC_250x250")', '').replace('googletag.display("google_dfp_MC_2x1,fluid")', '')
    clean_text = clean_text.replace('[', '').replace(']', '').replace('&amp', '').replace(', ■', '').replace('■ &lt', '').replace(';', '').replace('◆', '')
    return clean_text


def crawling_main_text(url):
        
        if type(url)==float:
            content = 'n/a'
        
        
        #조선일보 - 미완성
        elif 'chosun' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser")
            content_obj = soup.find_all("div", class_="article-body")
            
            
            content = clean_html(str(content_obj))
        
        
        #중앙일보
        elif 'joins' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser")

            content_obj = soup.find_all('div', class_="article_body")
            content = clean_html(str(content_obj))
        
        
        #동아일보
        elif 'donga' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser")

            content_obj = soup.find_all('div', class_="article_txt")
            content = clean_html(str(content_obj))
        
        
        #한겨례신문
        elif 'hani' in url:
            url = urllib.request.Request(url)
            res = urllib.request.urlopen(url).read().decode("utf-8")
    
            soup = BeautifulSoup(res, "html.parser")
            
            content_obj = soup.find_all('div', class_="text")
            content = clean_html(str(content_obj))
        

        #경향신문
        elif 'khan' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser")

            content_obj = soup.find_all('p', class_="content_text")
            content = clean_html(str(content_obj))
        
        
        #한국경제
        elif 'hankyung' in url:
            headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
            url = urllib.request.Request(url, headers=headers)
            res = urllib.request.urlopen(url).read().decode("utf-8")
    
            soup = BeautifulSoup(res, "html.parser")
    
            content_obj = soup.find_all('div', id="articletxt")
            content = clean_html(str(content_obj))
        
        
        #매일경제
        elif 'mk' in url:
            url = requests.get(url, headers={'User-Agent': 'Custom'})

            soup = BeautifulSoup(url.text, "html.parser", from_encoding='')
            soup_alt = bs4.BeautifulSoup(url.content.decode('euc-kr', 'replace'), "html.parser")

            content_obj = soup_alt.find_all('div', class_="art_txt")

            content = clean_html(str(content_obj))
    
    
        return content
                                     

In [9]:
#For Loop을 통한 본문 크롤링
#데이터 프레임에 추가

df_news['main_content']='n/a'

for i, url in enumerate(df_news['URL']):
    print(url)
    print(type(url))
    try:
        main_body = crawling_main_text(url)
        print(main_body)
        df_news['main_content'].iloc[i] = main_body
    except ConnectionError:
        continue
    

https://www.hankyung.com/economy/article/202204208696i
<class 'str'>
 LS그룹 계열의 도시가스업체 예스코홀딩스가 대신증권과 맥쿼리인프라를 비롯한 고배당주를 500억원어치 넘게 사들인 것으로 나타났다. 이 회사는 해외 스타트업(신생 벤처기업)에 공격적으로 투자했다가 700억원 넘는 손실을 입었다. 해외 투자를 접고 비교적 안정적 수익을 내는 고배당주를 비롯한 안전자산 투자에 집중하는 배경으로 해석된다.20일 투자은행(IB)업계에 따르면 예스코홀딩스는 지난 15일 기준 대신증권 지분 1.76%(89만4533주)를 보유 중이다. 작년 12월에 대신증권 주식을 184억1700만원을 들여 매입한 이후 최근까지 보유 중이다. 예스코홀딩스는 대신증권과 함께 고배당주로 꼽히는 맥쿼리인프라 주식 243만425주도 작년 9월에 316억원에 매입했다. 이 회사가 대신증권과 맥쿼리인프라를 매입한 배경은 고배당으로 해석된다. 대신증권은 지난해 결산배당으로 주당 1400원을 결정했다. 이 회사의 배당수익률은 6.7%로 작년 상장사 평균(1.52%)을 크게 웃돈다. 예스코홀딩스는 배당수익으로 대신증권 배당수익으로만 올해 12억원을 받는다. 맥쿼리인프라는 지난해 배당수익률은 5.3%에 달한다.예스코홀딩스는 자회사 예스코를 통해 비교적 부침이 없는 도시가스사업을 하고 있다. 서울시와 경기도 141만가구에 배관을 통해 도시가스를 공급하는 업체로 연간 200억~300억 원 안팎의 당기순이익을 올렸다. 이 회사가 고배당주를 500억원어치 넘게 매입한 것은 뼈아픈 투자 실패와 관계가 깊다.예스코홀딩스는 2016년 음성인식 기술업체인 에바오토메이션에 116억원, 싱가포르 음식료 배송 스타트업인 어니스트비에 198억을 각각 투자했다. LS가(家) 장손 구본웅 마음그룹 대표가 운영하던 미국 벤처캐피털(VC)인 포메이션8이 투자한 회사들이다. 하지만 투자업체들이 모두 사업에 실패하면서 투자금을 모두 날렸다. 예스코홀딩스는 월드비즈니스렌더스(WLC)에

/Users/ohikendoit/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


신화 기자, 무단전재 및 재배포 금지	부동산인프라' 지수를 기초 지수로 추종한다.다. 순자산이 2000억원을 넘어선 것은 2019년 7월 상장한 이후 2년9개월 만이다. 물가가 오르면서 실물자산 가치가 상승할 가능성이 있고, 변동성 장세에 리츠의 안정성이 상대적으로 부각되며 자금이 몰린 것으로 풀이된다. 연초 이후 개인이 364억원어치를 순매수했으며, 보험과 기타법인도 각각 150억원어치, 135억원어치를 순매수했다. 이 ETF는 상장 리츠 중에서 롯데리츠, ESR켄달스퀘어리츠, SK리츠, 제이알글로벌리츠 등 시가총액이 2000억원 이상인 우량 리츠와 맥쿼리인프라와 같은 인프라스트럭처 투자 회사에 분산 투자한다. 보유 기간에 발생하는 인컴수익에 초점을 맞춘 상품이다. 소액으로도 부동산, 인프라 등에 투자할 수 있다는 특징이 있다.
https://www.khan.co.kr/national/national-general/article/202204142115005
<class 'str'>
2015년부터 SK이노베이션·현대두산인프라코어 등서 재직한화생명에선 ‘고문’으로…“모종의 ‘방패막’ 활용 위해 발탁”, 김대기 대통령비서실장 내정자(사진)가 2015년부터 현재까지 민간기업 사외·감독이사 활동으로만 최소 6억4100만원의 보수를 받은 것으로 파악됐다. 김 내정자는 2015년부터 2018년 초까지는 SK이노베이션과 두산인프라코어(현 현대두산인프라코어)에서 사외이사를 겸임했는데, 이 기간에만 총합 4억원이 넘는 보수를 받았다. 김 내정자는 현재 한화생명과 맥쿼리한국인프라투융자회사(맥쿼리인프라), 두산중공업에서 각각 고문 및 사외이사를 맡고 있다. , 14일 경향신문 취재를 종합하면 이명박 정부의 대통령비서실 경제수석, 대통령 정책실장을 끝으로 2013년 2월 공직을 떠난 김 내정자는 그해 한화생명 고문을 시작으로 SK이노베이션, 두산인프라코어, 두산중공업, 한화생명, 맥쿼리인프라에서 사외·감독이사를 맡았다. , 2015년 3월 SK이노베이션 사외이사에 선임된 김 내정자


http://news.mk.co.kr/newsRead.php?no=292701&year=2022
<class 'str'>
차창희 기자, 무단전재 및 재배포 금지	당금을 지급했는데 현 주가 기준 연 5.5%로 배당수익률이 높다. 맥쿼리인프라의 연간 주당 배당금은 2017년 540원, 2018년 622원, 2019년 700원, 2020년 720원, 2021년 750원으로 증가하고 있다. 회복) 기조에 따라 위축된 유료 도로 통행량이 회복되면서 통행료 수입도 증가하고 있다.다.
https://www.khan.co.kr/economy/economy-general/article/202203311551001
<class 'str'>
지난해 고위공직자들의 평균 재산이 16억2000만원으로 1년 전에 비해 1억6000만원 가량 증가한 것으로 나타났다. 재산이 증가한 원인으로는 부동산과 더불어 주식 가격 상승이 꼽혔다. 특히, 재산 증가액 1~2위를 기록한 공직자들은 보유하고 있던 대규모 비상장주식 가치가 급등한 덕을 봤다. 수십억원 규모의 주식을 보유한 ‘큰 손’ 공직자 중에는 국내 주식 외에 미국, 중국, 베트남 등 해외 주식에 투자한 경우도 눈에 띄었다. 수백억대 비상장 주식으로 재산 증가 , 정부공직자윤리위원회는 31일 관보에 2022년도 정기재산변동 신고사항을 공개했다. 이번 재산공개 대상 공직자 1978명 중 재산규모 1위(350억6768만원으로)와 더불어 증가액도 1위를 기록한 이강섭 법제처장은 본인, 배우자, 차녀의 증권 재산으로 257억4677만원을 신고했다. 1년 전인 29억6500만원에서 9배 가량 증가했다. , 이 처장 본인이 보유하고 있는 한건(1만4000주)과 한겨레신문(200주)의 비상장주식이 99억9046만원으로 지난해 평가액(7억7450만원)에 비해 92억원가량 불어난 결과다. 배우자가 보유한 한건(1만5000주), 씨앤팜(128주), 올바이오텍(16주) 등 비상장주식의 가치도 9억3138만원에서 121억3903만원으로 112억원 증가했다. 한

	올해 주요 자산 중 투자 수익률이 가장 좋았던 상품은 원유인 것으로 나타났다. 원유에 투자하는 상장지수펀드(ETF)는 작년 한 해 70% 가까이 떨어졌다가 올해는 정반대로 70% 오르는 반전을 이뤄냈다. 코로나19 사태 이후 글로벌 경기 급랭으로 마이너스 수익률을 기록한 리츠(REITs·부동산투자회사)도 꾸준한 성과를 내며 반등했다.코로나19 변이 바이러스의 등장, 스태그플레이션(경기침체 속 물가 상승) 우려, 미국 중앙은행(Fed)의 긴축 움직임 등 삼중고가 덮친 가운데서도 미국 주식은 20% 넘게 오르며 지난해에 이어 건재한 모습을 과시했다. 반면 투자자들의 안전자산 선호로 지난해 강세를 보였던 채권과 금은 주요 자산군 중 수익률 최하위권으로 떨어졌다.  원유 1등…희귀 광물도 초강세올해 주요 자산별 ETF 수익률을 집계한 결과 서부텍사스원유(WTI) 가격을 따르는 ETF가 최고 성과를 낸 것으로 나타났다. 세계 최대 원유 ETF인 ‘미국오일펀드’(USO)는 올 들어 이달 23일까지 69.3%의 수익률을 기록했다. 이 ETF는 지난해 수익률이 -67.7%였지만 올해는 가장 극적 반전을 이룬 자산에 올랐다.국제 유가는 올해 50% 이상 급등했다. 연초 배럴당 50달러 안팎에 형성됐던 WTI 선물 가격은 꾸준히 올라 10월엔 80달러 선을 돌파했다. 최근에는 주요 산유국이 석유 증산 방침을 유지하기로 한 데다 오미크론 변수 부각으로 수요 둔화 우려가 퍼지며 70달러 초반 선에서 거래되고 있다. 올해 유가 상승을 이끈 것은 세계 경제 회복 기대감이었다. 주요 투자은행(IB)들은 연말 코로나19 변이 바이러스 확산으로 수요가 영향을 받을 수 있다는 우려에도 내년 유가가 배럴당 100달러에 이를 것으로 전망하고 있다. JP모간은 브렌트유가 내년 배럴당 125달러를 넘어설 것으로 내다봤다. 현재 수준보다 60% 이상 오를 것이란 관측이다.구리 ETF도 올해 25.7% 성과를 내 상위권에 들었다. ‘미국구리인덱스펀드’(CPER)는 작년에 이어 올해도 20% 넘는 수익

...

여전히 존재하고 있는 경기 불안감 속에서도 개별 종목 장세는 이어지고 하루에도 급등을 보이는 종목들이 적지 않게 보여지고 있다. 그러나, 대부분의 개인투자자들은 이러한 상승세를 즐기지 못하고 대부분인 최근 지수 급락으로 인한 손실에서 헤어나지 못한 채 가슴 아픈 손실만 끌어 안고 있는 현실이다.“주투연”의 시황분석에 따르면 미국 소비지출 지표의 개선으로 코스피 지수는 1.5% 상승했다. 특히 삼양옵틱스 피에스엠씨 ad모터스 모나미 코엔텍 에스코넥 jyp ent. 미래산업 바른손 위노바 파루 로엔 유성티엔에스 stx조선해양 지아이바이오 클루넷 휴비츠 네오위즈인터넷 등이 상승했지만, 대현 하이닉스 우리들생명과학 아인스MM 대주산업 에듀박스 웰크론 lg디스플레이 엠텍비젼 은 하락했다.주투연 바로가기 ====&gtwww.w-bincent.com 이런 시황 속에서 개인투자자들의 경우 매번 무리한 추격매수와 투매로 인해 큰 손실을 반복하고 있는데, 혼자서 수익을 내기 어려운 개인들은 “주투연” 을 방문해 도움을 받아 보는것이 좋을 것이다.5년간 수많은 방송출신 전문가들을 배출하고 92%의 최고 적중률을 자랑하는 투자전문가 정우진 CEO가 운영하는 주투연 전문가 그룹은 따라잡지 못할 저가주를 마치 급등과 대박을 안겨줄 처럼 광고하는 비현실성을 배제하고 철저한 기업분석과 날카로운 판단력으로 직장인이나 초보자도 쉽게 매매가능한 시스템을 완성하여 메디포스트(105%),대현(112%),쌍방울트라이(75%),오로라(112%),동아화성(114%),큐로컴(135%),뷰웍스(83%),동일금속(95%),안철수연구소(50%),이수화학(76%),대창(62%),원익쿼츠(76%),이니시스(51%)등 높은 수익률은 물론, 올 초부터 현재 457추천 411수익의 89% 적중률과 같은 기간 단기프로젝트를 통해 114종목 추천 108종목 수익 이라는 어떤 공간도 흉내내지 못하는 92%라는 놀라운 적중률을 기록하는 노하우를 발휘하고 있다.이제 ‘주투연’을 방문해 95%라는 일반투자자들의 손해라는 통계 속

▶ 2NE1 박봄 과거 '통통-청순' 지금보다 더 예뻐?▶ 게임 '덕후' 여대생, 연 300억 매출 여사장 되다▶ 김종서, "임재범에 박탈감 느껴" 깜짝 고백… 마포대교 보컬연습 ▶ 양진선-기성용, 다정한 커플사진 연인관계?▶ 화보 메간폭스, No보톡스 No성형 공개, 무단전재 및 재배포 금지	2009년 7.6%, 2010년 6.7%에 달한다.최근 3년간 현금배당 성향은 104.1%에 이른다. 높은 배당 성향은 현행법상의 공제 혜택 덕분이다. 자본시장법상 투자회사는 배당가능이익 가운데 90% 이상을 주주에게 배당하면 당해 연도 과표에서 배당금을 공제받을 수 있다.
http://news.mk.co.kr/news_forward.php?no=447959&year=2011
<class 'str'>
▶ 2NE1 박봄 과거 '통통-청순' 지금보다 더 예뻐?▶ 게임 '덕후' 여대생, 연 300억 매출 여사장 되다▶ 김종서, "임재범에 박탈감 느껴" 깜짝 고백… 마포대교 보컬연습 ▶ 양진선-기성용, 다정한 커플사진 연인관계?▶ 화보 메간폭스, No보톡스 No성형 공개, 무단전재 및 재배포 금지	2009년 7.6%, 2010년 6.7%에 달한다.최근 3년간 현금배당 성향은 104.1%에 이른다. 높은 배당 성향은 현행법상의 공제 혜택 덕분이다. 자본시장법상 투자회사는 배당가능이익 가운데 90% 이상을 주주에게 배당하면 당해 연도 과표에서 배당금을 공제받을 수 있다.
http://www.hani.co.kr/arti/society/area/483728.html
<class 'str'>
 ‘돈 먹는 하마’로 불리는 광주 제2순환도로를 공채로 사들이는 방안을 광주시가 추진한다.강운태 광주시장은 20일 간부회의에서 “제2순환도로에 해마다 세금 200억원을 지원하는 것보다는 저리로 공채를 발행해 민자시설 구간을 매입하는 방안을 추진할 필요가 있다”고 밝혔다.강 시장은 “공채 발행으로 시 부채가 늘어나더라도 제2순환도로 보전금 문제를 해결한다면 시민들이 이해할 것”이라며 “기획실장이 전담

민선5기 광역단체장 연속 인터뷰강운태 광주시장“시민이 행복한 광주를 만들겠습니다.”강운태 광주시장이 당선 이후 입버릇처럼 강조하는 민선 5기 시정 목표다. 그는 어느 석상이든 시민의 행복과 시민의 만족을 화두로 던졌다. 버스노선을 바꾸든지 도시계획을 세우든지 행정의 수요자인 시민의 행복에 초점을 맞추겠다고 다짐했다. 공무원들한테도 이 목표를 향해 현장을 열심히 뛰고, 새로운 아이디어를 내라고 채근했다.-시정 구호를 ‘행복한 창조도시 광주’로 제시했다. 창조도시의 개념은 무엇인가?“시민이 살아가는 삶의 방식 전반이 다른 지역에 비해 앞서가는 도시를 만들겠다. 구체적으로는 광주의 정신, 문화, 제도, 상품, 도시경영이 대한민국은 물론 세계의 모델이 되도록 한다는 구상이다.”-당선 이후 ‘시민과의 소통’을 유난히 강조했는데. 너무 당연한 이야기라서 배경이 궁금하다.“현안사업이 반목과 대립으로 차질을 빚은 사례가 많았다. 옛 전남도청 별관 보존, 야구 돔구장 건설, 관광레저타운 조성, 광주공항 기능 이전 등이 대표적이다. 소통의 부재에 따른 것인 만큼 주요 사업의 기획·집행·평가 때 단계별로 시민 의견을 듣겠다. 매주 금요일 열리는 시민과의 만남, 각계각층이 참여하는 광주공동체원탁회의 등 통로를 마련하겠다. ”-보 건설과 준설을 통한 4대강 사업을 반대한다면 영산강은 어떤 방식으로 살리겠는가?“보는 천천히 막아도 된다. 보를 막는 것이 좋은지 타당성 조사를 해보자는 얘기다. 그 전에 수질개선에 중점을 두고, 광주천을 비롯한 소하천을 먼저 살려야 한다.”-광주천 수질은 어떻게 개선하나. 재원은?“우수와 오수를 따로 처리하는 분류식 하수관거를 설치하겠다. 현재 추진중인 예산사업 73㎞, 임대형 민간투자사업 250㎞를 서둘러 추진하겠다. 2012년까지 하수처리장의 방류 수질을 현재 생화학적 산소요구량 10㎎/ℓ에서 5㎎/ℓ로 만들겠다.”-개방형 직위를 늘리겠다고 했는데?“인권담당관은 민주·인권·평화도시로서 정체성을 높이고, 인권지수를 개발하고 실천하는 책임을 지는 업무를 


http://news.mk.co.kr/news_forward.php?no=472809&year=2009
<class 'str'>

nan
<class 'float'>
n/a
http://news.khan.co.kr/kh_news/khan_art_view.html?artid=20090406002280
<class 'str'>

http://news.khan.co.kr/kh_news/khan_art_view.html?artid=200904061715555&code=960801
<class 'str'>
KBS1 &lt시사기획 쌈&gt은 7일 오후 10시 ‘세금 먹는 하마’로 불리는 민자사업의 구조적인 문제점을 파헤치는 ‘황금알 민자사업, 1부 맥쿼리의 세금방패’를 방송한다. , 민자사업은 도로와 교량, 터널 등 사회간접자본 시설을 나랏돈이 아닌 민간자본으로 만든다는 사업이다. 민간의 창의성과 효율성을 살려 다리를 놓고 도로를 닦자는 취지로 시작됐지만 현실은 어둡기만 한 경우가 많다. , 이에 제작진은 주요 민자사업의 실태와 대안 등을 조명한다. 특히 민자사업의 선구자라는 한국맥쿼리인프라투융자회사에 주목한다. 맥쿼리인프라는 2조원에 이르는 자금을 굴리는 국내 민자사업의 가장 큰 투자 회사다. 실제로 맥쿼리인프라는 2002년 회사가 세워진 뒤 우면산터널과 인천공항고속도로, 마창대교, 광주순환도로 등 전국 15개 도로와 교량, 터널 등에 투자를 해왔다. 지난해 당기순이익만 1837억원을 올렸다. , 맥쿼리인프라가 100% 지분을 갖고 있는 광주순환도로 1구간의 경우 당초 광주시와 맺은 협약서상으로는 2006년부터 수익을 내고, 그에 따른 법인세를 내는 것으로 돼 있었다. 그러나 광주순환도로 1구간은 지금까지 계속 적자 상태로 법인세를 낸 적이 한 번도 없다. 비싼 통행료와 정부 보조금까지 지원받는 알짜 사업을 벌이고 있는데도 적자를 벗어나지 못하고 있는 이유는 무엇일까. 그 비밀은 사채 수준의 높은 이자에 있다. 
http://news.mk.co.kr/news_forw

 호주계 사모펀드(PEF)가 11개 자회사를 거느린 국내 에너지업계의 큰손 SK ES의 지분 49%를 인수한다. 호주 맥쿼리은행의 자회사인 맥쿼리 코리아 오퍼튜니즈운용㈜의 데이비드 러셋(사진) 대표는 9일 본지와의 인터뷰에서 "가스.전력 등에 투자하는 새로운 PEF인 '맥쿼리 코리아 오퍼튜니티즈 사모투자전문회사'(MKOF)를 출범시켜 SK ES의 지분을 인수한다"고 밝혔다. SK ES는 LPG 회사인 SK가스.대한도시가스.부산도시가스 등 9개 도시가스사, 열병합발전회사인 익산에너지를 거느리고 있는 회사로 국내 도시가스 공급시설 시장에서 25%의 점유율을 차지하고 있다. 러셀 대표는 "MKOF는 국내 투자자들로부터 3400억원의 출자약정을 받았으며, 앞으로 1조원 이상으로 증자해 국내 가스.전력.상수도.미디어.통신 등에 집중 투자할 계획"이라고 밝혔다. 현재 국내 증시 상장을 예정하고 있는 국내 최대의 인프라 펀드인 '맥쿼리 한국인프라투융자회사'(MKIF)가 유료도로.터널.교량 등 '사회기반시설에 대한 민간투자법'(민투법) 이 규정하고 있는 시설물에 대해 투자한다면 MKIF는 민투법 이외의 인프라 자산에 투자하는 방식이다. 러셀 대표는 "이미 신한.우리은행과 대한생명, 서울대 발전기금 등이 출자를 약속했고, 일본계 4개 은행과도 협의를 진행 중"이라며 "우선 국내 투자자 위주로 모집을 하되 장기적인 투자성향을 가진 투자자라면 외국 자본도 참여를 허용할 방침"이라고 강조했다. 그는 미디어 부문에 대한 투자 계획도 밝혔다. 현재 유선방송사 몇 곳과 투자 협상을 벌이고 있고, 방송을 위한 기지국, 송신탑, 중계탑 등 방송 관련 인프라에도 투자할 계획이라고 말했다. 러셀 대표는 "매쿼리는 최근 5년간 인프라 펀드라는 새로운 블루오션을 개척해 투자자들에게 연평균 19.4%의 복리수익을 안겨주고 있다"며 "경제 전망이 밝은 만큼 한국은 매우 중요한 시장"이라고 밝혔다.윤창희 기자 theplay@joongnag.co.kr
https://www.donga.com/news/ar

In [10]:
#Export dataframe if necessary
df_news.to_excel("df_news_updated.xlsx")

  
  
  
  
  
## Reference Code for Individual Media Source  (Legacy; No Longer Updated)

### 한겨례

In [11]:
import urllib.request
from bs4 import BeautifulSoup

def contents_hankyoreh():
    
    #샘플 기사 URL
    list_url = "https://www.hani.co.kr/arti/politics/politics_general/1038853.html"
    
    url = urllib.request.Request(list_url)
    res = urllib.request.urlopen(url).read().decode("utf-8")
    #print(res)
    
    soup = BeautifulSoup(res, "html.parser")
    params1 = []
    params2 = []
    
    for link1, link2 in zip(soup.find_all('p', class_="date-time"), soup.find_all('div', class_="text")):
        params1.append(link1.get_text())
        params2.append(link2.get_text())
        
    for i1, i2 in zip(params1, params2):
        #print(i1.strip(), end=' ')
        print(i2.strip())
        
contents_hankyoreh()
    

맥쿼리한국인프라투융자회사의 김대기 감독이사(청와대 비서실장 내정자) 소개 화면. 맥쿼리인프라 누리집 갈무리



김대기 대통령 비서실장 내정자가 13일 현재 민자사업 투자를 주력으로 하는 맥쿼리한국인프라투융자회사(맥쿼리인프라)에서 이사로 재직 중인 것으로 확인됐다. 맥쿼리인프라는 도로·철도 민자사업에 투자해 투자금을 회수하고 그 과정에서 국민 부담을 가중시키는 대표적인 외국자본이다. 이명박 정부 시절 통계청장, 문화체육관광부 2차관, 대통령실 경제수석과 정책실장을 두루 역임한 김 내정자는 또 한화생명 고문과 두산중공업 등의 사외이사도 맡고 있는 것으로 드러났다.

<한겨레> 취재를 종합하면 김 내정자는 2020년 7월28일에 맥쿼리인프라의 감독이사로 선임돼 올해 3월 재선임됐다. 맥쿼리인프라는 도로·도시가스·철도 등 공공이 발주하는 인프라 사업에 투자해 이익을 가져가는 상장 펀드다. 맥쿼리는 이명박 정부 시절인 2008년 12월 서울지하철 9호선 사업에 투자했고 최소운영수입보장 조건에 따라 서울시로부터 매년 수백억원씩을 챙겼다. 2013년에는 9호선 요금 인상까지 결정하면서 서울시와 갈등을 빚다가 철수했다. 이런 식으로 정부·지방자치단체와 소송이 잦아 맥쿼리가 퇴직관료들을 ‘방패막이’로 영입하고 있다는 비판을 받기도 했다. 김 내정자는 기획예산처 예산실장 출신으로 공직에서의 업무 경력이 민자 인프라 사업과도 밀접하다.
김 내정자가 2020년 감독이사로 온 뒤 맥쿼리인프라는 부산신항 제2배후도로, 해양에너지·서라벌도시가스, 인천김포고속도로 등에 6천억원 규모 투자를 결정했다. 지난해엔 광주 제2순환도로 민자사업 담당 관청인 광주시를 상대로 대한상사중재원에 중재를 신청해 49억원을 받아내기도 했다.

맥쿼리 쪽은 “감독이사는 일반 기업의 사외이사와 비슷하게 회사의 투자 업무를 관리·감독하는 역할”이라며 “김 내정자도 다른 감독이사들의 추천을 받아 주주총회를 통해 선임한 것”이라고 했다. 김 내정자는 맥쿼리인프라에서 매년 7천만원의 보수를 받았다고 한다. 김 내정자

### 한국경제

In [12]:
#한국경제와 같은 경우 

import re
import urllib.request
from bs4 import BeautifulSoup

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>|\s{2}|\n')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def contents_hankyung():
    
    #샘플 기사 URL
    list_url = "https://www.hankyung.com/finance/article/2021122646691"
    
    url = urllib.request.Request(list_url, headers={'User-Agent': 'Custom'}) #
    res = urllib.request.urlopen(url).read().decode("utf-8")
    
    soup = BeautifulSoup(res, "html.parser")
    
    content_obj = soup.find_all('div', id="articletxt")
    content = cleanhtml(str(content_obj))
    return content
        

contents_hankyung()
    

'[\t올해 주요 자산 중 투자 수익률이 가장 좋았던 상품은 원유인 것으로 나타났다. 원유에 투자하는 상장지수펀드(ETF)는 작년 한 해 70% 가까이 떨어졌다가 올해는 정반대로 70% 오르는 반전을 이뤄냈다. 코로나19 사태 이후 글로벌 경기 급랭으로 마이너스 수익률을 기록한 리츠(REITs·부동산투자회사)도 꾸준한 성과를 내며 반등했다.코로나19 변이 바이러스의 등장, 스태그플레이션(경기침체 속 물가 상승) 우려, 미국 중앙은행(Fed)의 긴축 움직임 등 삼중고가 덮친 가운데서도 미국 주식은 20% 넘게 오르며 지난해에 이어 건재한 모습을 과시했다. 반면 투자자들의 안전자산 선호로 지난해 강세를 보였던 채권과 금은 주요 자산군 중 수익률 최하위권으로 떨어졌다.  원유 1등…희귀 광물도 초강세올해 주요 자산별 ETF 수익률을 집계한 결과 서부텍사스원유(WTI) 가격을 따르는 ETF가 최고 성과를 낸 것으로 나타났다. 세계 최대 원유 ETF인 ‘미국오일펀드’(USO)는 올 들어 이달 23일까지 69.3%의 수익률을 기록했다. 이 ETF는 지난해 수익률이 -67.7%였지만 올해는 가장 극적 반전을 이룬 자산에 올랐다.국제 유가는 올해 50% 이상 급등했다. 연초 배럴당 50달러 안팎에 형성됐던 WTI 선물 가격은 꾸준히 올라 10월엔 80달러 선을 돌파했다. 최근에는 주요 산유국이 석유 증산 방침을 유지하기로 한 데다 오미크론 변수 부각으로 수요 둔화 우려가 퍼지며 70달러 초반 선에서 거래되고 있다. 올해 유가 상승을 이끈 것은 세계 경제 회복 기대감이었다. 주요 투자은행(IB)들은 연말 코로나19 변이 바이러스 확산으로 수요가 영향을 받을 수 있다는 우려에도 내년 유가가 배럴당 100달러에 이를 것으로 전망하고 있다. JP모간은 브렌트유가 내년 배럴당 125달러를 넘어설 것으로 내다봤다. 현재 수준보다 60% 이상 오를 것이란 관측이다.구리 ETF도 올해 25.7% 성과를 내 상위권에 들었다. ‘미국구리인덱스펀드’(CPER)는 작년에 이어 올해도 20% 넘는

### 경향신문

In [13]:
#샘플 기사 URL
list_url = 'https://www.khan.co.kr/national/national-general/article/202204142115005'

def contents_kyunghyang(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser")

    content_obj = soup.find_all('p', class_="content_text")

    content = cleanhtml(str(content_obj))
    return content

contents_kyunghyang(list_url)

'[2015년부터 SK이노베이션·현대두산인프라코어 등서 재직한화생명에선 ‘고문’으로…“모종의 ‘방패막’ 활용 위해 발탁”, 김대기 대통령비서실장 내정자(사진)가 2015년부터 현재까지 민간기업 사외·감독이사 활동으로만 최소 6억4100만원의 보수를 받은 것으로 파악됐다. 김 내정자는 2015년부터 2018년 초까지는 SK이노베이션과 두산인프라코어(현 현대두산인프라코어)에서 사외이사를 겸임했는데, 이 기간에만 총합 4억원이 넘는 보수를 받았다. 김 내정자는 현재 한화생명과 맥쿼리한국인프라투융자회사(맥쿼리인프라), 두산중공업에서 각각 고문 및 사외이사를 맡고 있다. , 14일 경향신문 취재를 종합하면 이명박 정부의 대통령비서실 경제수석, 대통령 정책실장을 끝으로 2013년 2월 공직을 떠난 김 내정자는 그해 한화생명 고문을 시작으로 SK이노베이션, 두산인프라코어, 두산중공업, 한화생명, 맥쿼리인프라에서 사외·감독이사를 맡았다. , 2015년 3월 SK이노베이션 사외이사에 선임된 김 내정자는 같은 시기 두산인프라코어 사외이사에도 선임됐다. 두 회사에서 2018년 3월 무렵까지 사외이사로 일했다. 2019년 3월에는 두산중공업 사외이사로 선임됐다. 이후 지난달 29일 주주총회에서 재선임돼 현재까지 두산중공업 사외이사로 재직 중이다. , 김 내정자가 SK이노베이션, 두산인프라코어, 두산중공업에서 받은 보수는 기업들이 사업보고서에 공시한 사외이사들의 ‘1인당 평균 보수액’을 통해 유추할 수 있다. , 김 내정자는 2015년부터 2017년까지 3년간 SK이노베이션과 두산인프라코어에서 총합 4억1000만원 정도를 받은 것으로 파악된다. SK이노베이션의 사업보고서에 따르면, 2015년부터 2017년까지 사외이사 감사위원에게 지급된 1인당 평균 보수액은 총합 2억4000만원 선이다. 같은 시기 두산인프라코어가 사외이사 감사위원에게 지급한 금액은 약 1억7000만원이다. 이어 2019년부터 2021년까지 두산중공업에서 약 1억6100만원을 받은 것으로 보인다. , 김 내정자는 2

### 동아일보

In [14]:
list_url = 'https://www.donga.com/news/article/all/20220404/112693737/1'

def contents_donga(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser")

    content_obj = soup.find_all('div', class_="article_txt")

    content = cleanhtml(str(content_obj))
    return content

contents_donga(list_url)

'[주가지수 10% 이상 내려 투자적기유가-美금리 상승에 시장은 불안… ELS-리츠 등으로 위험 분산을은퇴세대엔 코코본드 인기 높아져… 연 3~4% 수익률 3개월마다 배당크게보기정현석 신한PWM SFC강남센터 PB팀장Q. 은퇴자 A 씨는 최근 거치식 보험의 만기가 돌아왔다는 연락을 받았다. 또 안전하다고 믿었던 공모주 펀드는 일부 손실이 난 채로 환매됐다. 보험 만기 환급금과 펀드 환매금으로 5억 원의 목돈을 쥐게 된 것이다. 하지만 우크라이나 전쟁과 미국 금리 인상 등으로 금융시장이 불안한 상황에서 이를 어떻게 운용할지 고민이 많다. A. 연초부터 금융시장의 움직임이 심상치 않다. 올해 초 국제통화기금(IMF)은 세계 경제성장률을 4.4%로 전망했지만 이마저도 다시 낮출 것이라는 예상이 지배적이다. 높은 인플레이션과 우크라이나 사태로 인한 원자재 가격 급등, 공급망 차질 장기화 등이 원인이다. 지난해 배럴당 60달러 수준이던 유가는 100달러 이상으로 치솟았고 미국은 3년여 만에 지속적인 금리 인상을 예고했다. 그렇다면 올해는 어떻게 포트폴리오를 구성해야 할까. 안정적이고 균형 잡힌 포트폴리오를 구성하기 위해선 금리 인상기에 인플레이션 헤지(위험 회피)가 가능한 자산, 유동성을 쉽게 확보할 수 있는 자산 등으로 포트폴리오를 구성하는 게 바람직하다.  시장 변동성이 커지면서 최근 주가연계증권(ELS)의 매력이 커졌다. 특히 주가지수와 연계한 지수형 ELS가 부담이 적다. 연초에 비해 주요국 지수가 10% 넘게 하락해 기준가격이 상대적으로 낮아졌기 때문이다. 만기 3년으로 6개월마다 조기 상환 기회를 주는 스텝다운형 구조를 선택하면 상환 확률은 더 높아진다. 다만 가입할 때 ‘녹인(Knock-in)’ 구조인지, ‘노(No) 녹인’ 구조인지 살펴봐야 한다. 녹인은 가입 기간에 한 번이라도 기초자산이 손실 구간 아래로 내려가면 원금 손실이 날 수 있다. 지수가 50% 이상 하락한 적이 없으면 녹인 구조가 수익률 측면에서 유리할 수 있지만 그 반대라면 주의해야 한다.

### 중앙일보

In [15]:
list_url = 'https://news.joins.com/article/2137057'

def contents_joongang(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser")

    content_obj = soup.find_all('div', class_="article_body")

    content = cleanhtml(str(content_obj))
    return content

contents_joongang(list_url)

"[금융감독원은 7일 신한금융그룹과 호주 최대 투자 은행인 맥쿼리 은행이 공동 설립한 '맥쿼리코리아 인프라스트럭처 펀드(MKIF)'가 서울과 런던 증시에 동시 상장될 예정이라고 발표했다. MKIF는 상장을 통해 약 5000억원의 신규 자금을 조달할 계획이며, 3월이면 증시에서 개인투자자들도 인프라 펀드를 사고팔 수 있다. MKIF의 보유 자산은 천안~논산 고속도로, 인천공항 고속도로(지분 25%), 지하철 9호선 등으로 총 1조2600억원에 달한다.]"

### 매일경제

In [16]:
import bs4
list_url = 'https://www.mk.co.kr/news/stock/view/2022/04/339384/'

def contents_maeil(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser", from_encoding='')
    soup_alt = bs4.BeautifulSoup(url.content.decode('euc-kr', 'replace'), "html.parser")

    content_obj = soup_alt.find_all('div', class_="art_txt")

    content = cleanhtml(str(content_obj))
    return content

contents_maeil(list_url)

'[미래에셋자산운용이 \'TIGER 리츠부동산인프라\' 상장지수펀드(ETF)의 순자산이 2000억원을 돌파했다고 15일 밝혔다.\r미래에셋에 따르면 전일 종가 기준으로 이 ETF의 순자산이 2111억원으로 집계됐다. 순자산이 2000억원을 넘어선 것은 2019년 7월 상장한 이후 2년9개월 만이다. 물가가 오르면서 실물자산 가치가 상승할 가능성이 있고, 변동성 장세에 리츠의 안정성이 상대적으로 부각되며 자금이 몰린 것으로 풀이된다. 연초 이후 개인이 364억원어치를 순매수했으며, 보험과 기타법인도 각각 150억원어치, 135억원어치를 순매수했다. 이 ETF는 상장 리츠 중에서 롯데리츠, ESR켄달스퀘어리츠, SK리츠, 제이알글로벌리츠 등 시가총액이 2000억원 이상인 우량 리츠와 맥쿼리인프라와 같은 인프라스트럭처 투자 회사에 분산 투자한다. 보유 기간에 발생하는 인컴수익에 초점을 맞춘 상품이다. 소액으로도 부동산, 인프라 등에 투자할 수 있다는 특징이 있다.\r또한 해당 ETF는 \'에프앤가이드 리츠부동산인프라\' 지수를 기초 지수로 추종한다.\r[신화 기자][ⓒ 매일경제 &amp; mk.co.kr, 무단전재 및 재배포 금지]\twindow.jQuery || document.write("")]'

### 조선일보

In [17]:
import re
from bs4 import BeautifulSoup

list_url = 'https://www.chosun.com/economy/stock-finance/2021/12/10/OYYDOUAR35HZHBIOO5YAL4XTD4/?utm_source=bigkinds&utm_medium=original&utm_campaign=news'

content_obj = []

def contents_chosun(list_url):
    
    url = requests.get(list_url, headers={'User-Agent': 'Custom'})

    soup = BeautifulSoup(url.text, "html.parser")
    #content_obj = soup.find_all('section', class_="article-body")
    for tag in soup.select("div", class_="article-body"):
        print(tag)
        #write_ws.append([tag.text])
    
    content = cleanhtml(str(content_obj))
    return content

contents_chosun(list_url)

<div id="fusion-app"><div data-fusion-collection="layouts" data-fusion-message="Could not render component [layouts:article]" data-fusion-type="article" id="article" style="display:none"></div></div>
<div data-fusion-collection="layouts" data-fusion-message="Could not render component [layouts:article]" data-fusion-type="article" id="article" style="display:none"></div>


'[]'